## Задача 1. Функция распределения экспериментов по бакетам

Реализуйте функцию add_experiment, отвечающую за распределение экспериментов по бакетам

In [18]:
import pandas as pd
import numpy as np


In [15]:
def add_experiment(experiment, buckets):
    """Проверяет можно ли добавить эксперимент, добавляет если можно.

    Распределять эксперименты нужно так, чтобы умещалось как можно больше экспериментов.

    :param experiment (dict): параметры эксперимента, который нужно запустить.
        Ключи словаря:
            - id - идентификатор эксперимента.
            - buckets_count - необходимое количество бакетов.
            - conflicts - список идентификаторов экспериментов, которые нельзя проводить
                одновременно на одних и тех же пользователях.
    :param buckets (list[list[int]]): список бакетов, в каждом бакете перечислены
            идентификаторы экспериментов, которые в нём проводятся.

    :return (success, buckets):
        success (boolean) - можно ли добавить эксперимент, True - можно, иначе - False
        buckets (list[list[int]]) - обновлённый список бакетов с добавленным экспериментом,
            если эксперимент добавить можно.
    """
    
    if experiment['buckets_count'] > len(buckets):
        return False, buckets
    
    else:
        # список номеров доступных бакетов (где не лежат конфликты)
        available_buckets_list = []
        
        # подсчет кол-ва доступных бакетов
        for i in range(len(buckets)):
            if len(list(set(buckets[i]) & set(experiment['conflicts']))) == 0:
                available_buckets_list.append(i)
        
        # если доступных бакетов меньше, чем требуется для теста, возвращаем false
        if len(available_buckets_list) < experiment['buckets_count']:
            return False, buckets
        
        # если доступных бакетов достаточно, пробуем распихать их по доступным бакетам
        buckets_count = experiment['buckets_count']
        for i in available_buckets_list:
            if buckets_count <= 0:
                break
            buckets[i].append(experiment['id'])
            buckets_count -= 1
                    
        
    return True, buckets
            
    
    

In [17]:
total_buckets_count = 4
buckets = [[] for _ in range(total_buckets_count)]
success, buckets = add_experiment({'id': 0, 'buckets_count': 5, 'conflicts': []}, buckets)
print(success, buckets)
# для эксперимента необходимо больше бакетов, чем доступно (5 > 4)
# success, buckets = False, [[], [], [], []]

success, buckets = add_experiment({'id': 1, 'buckets_count': 4, 'conflicts': [4]}, buckets)
print(success, buckets)
# success, buckets = True, [[1], [1], [1], [1]]

success, buckets = add_experiment({'id': 2, 'buckets_count': 2, 'conflicts': [3]}, buckets)
print(success, buckets)
# эксперимент с id=2 может быть в любых двух бакетах их четырёх
# success, buckets = True, [[1, 2], [1], [1, 2], [1]]

success, buckets = add_experiment({'id': 3, 'buckets_count': 2, 'conflicts': [2]}, buckets)
print(success, buckets)
# можем добавить в бакеты, где не запущен экперимент с id=2
# success, buckets = True, [[1, 2], [1, 3], [1, 2], [1, 3]]

success, buckets = add_experiment({'id': 4, 'buckets_count': 1, 'conflicts': [1]}, buckets)
print(success, buckets)
# не можем добавить, так как во всех бакетах запущен эксперимент, с которым конфликт
# success, buckets = False, [[1, 2], [1, 3], [1, 2], [1, 3]]

False [[], [], [], []]
True [[1], [1], [1], [1]]
True [[1, 2], [1, 2], [1], [1]]
True [[1, 2], [1, 2], [1, 3], [1, 3]]
False [[1, 2], [1, 2], [1, 3], [1, 3]]


## Задача 2. Функция распределения пользователей по бакетам и группам

Реализуйте функцию process_user, отвечающую за распределение пользователей по бакетам

In [20]:
import hashlib


def get_hash_modulo(value: str, modulo: int, salt: str):
    """Вычисляем остаток от деления: (hash(value + salt)) % modulo."""
    hash_value = int(hashlib.md5(str.encode(value + salt)).hexdigest(), 16)
    return hash_value % modulo

def process_user(user_id, buckets, experiments, bucket_salt):
    """Определяет в какие эксперименты попадает пользователь.

    Сначала нужно определить бакет пользователя.
    Затем для каждого эксперимента в этом бакете выбрать пилотную или контрольную группу.

    :user_id (str): идентификатор пользователя
    
    :buckets (list[list[int]]): список бакетов c id-шками тестов внутри
    
    :experiments (list[dict]): список словарей {id = идентификатор эксперимента, salt = соль для сплита внутри теста}
    
    :bucket_salt (str): соль для разбиения пользователей по бакетам.
    
    :return bucket_id, experiment_groups:
        - bucket_id (int) - номер бакета (индекс элемента в buckets)
        - experiment_groups (list[tuple]) - список пар: id эксперимента, группа.
            Группы: 'A', 'B'.
        Пример: (8, [(194, 'A'), (73, 'B')])
    """
    
    modulo = len(buckets)
    bucket_id = get_hash_modulo(user_id, modulo, bucket_salt)
    
    exps = buckets[bucket_id] # эксперименты, в которые попал пользователь
    exps_dict = [] # список, в который кладется связка "id экспа + группа экспа"

    # словарь перевода хэша в название группы
    groups_dict = {
        0: 'B',
        1: 'A'
    }
    
    # 
    for id_ in exps:
        id_test_salt = ''
        for item in experiments:
            if item['id'] == id_:
                id_test_salt = item['salt']
                break
        
        group = groups_dict[get_hash_modulo(user_id, 2, id_test_salt)]
        exps_dict.append((id_, group))
        
    return bucket_id, exps_dict
    

In [26]:
user_id = '1001'
experiments = [{'id': 0, 'salt': '0'}, {'id': 1, 'salt': '1'}]
buckets = [[0, 1], [1], []]
bucket_salt = 'a2N4'
bucket_id, experiment_groups = process_user(user_id, buckets, experiments, bucket_salt)

print(bucket_id, experiment_groups)

# В зависимости от значений bucket_salt и солей экспериментов, можно получить один из вариантов:

# bucket_id, experiment_groups = 0, [(0, 'A'), (1, 'A')]
# bucket_id, experiment_groups = 0, [(0, 'A'), (1, 'B')]
# bucket_id, experiment_groups = 0, [(0, 'B'), (1, 'A')]
# bucket_id, experiment_groups = 0, [(0, 'B'), (1, 'B')]
# bucket_id, experiment_groups = 1, [(1, 'A')]
# bucket_id, experiment_groups = 1, [(1, 'B')]
# bucket_id, experiment_groups = 2, []

0 [(0, 'B'), (1, 'B')]
